In [1]:
!pip install plotly==4.8
import pandas as pd
import numpy as np
import plotly.express as px
print('modules are imported')

     |████████████████████████████████| 11.5MB 2.8MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
modules are imported


load dataset

In [2]:
dataset_url='https://raw.githubusercontent.com/datasets/covid-19/master/data/countries-aggregated.csv'
df=pd.read_csv(dataset_url)

check dataframe

In [3]:
df.head()

,Date,Country,Confirmed,Recovered,Deaths
0,2020-01-22,Afghanistan,0,0,0
1,2020-01-23,Afghanistan,0,0,0
2,2020-01-24,Afghanistan,0,0,0
3,2020-01-25,Afghanistan,0,0,0
4,2020-01-26,Afghanistan,0,0,0


In [4]:
df.tail()

,Date,Country,Confirmed,Recovered,Deaths
50458,2020-10-10,Zimbabwe,8010,6492,230
50459,2020-10-11,Zimbabwe,8011,6504,230
50460,2020-10-12,Zimbabwe,8021,7627,230
50461,2020-10-13,Zimbabwe,8036,7632,230
50462,2020-10-14,Zimbabwe,8055,7640,231


In [5]:
df.shape

(50463, 5)

## pre-processing data

In [6]:
df=df[df.Confirmed > 0]
df_sort=df[df.Confirmed > 0].sort_values(by =['Date']); 
#df[df.Country=='China']
df_sort.head()

,Date,Country,Confirmed,Recovered,Deaths
23229,2020-01-22,Japan,2,0,0
46725,2020-01-22,US,1,0,0
44322,2020-01-22,Taiwan*,1,0,0
45123,2020-01-22,Thailand,2,0,0
24297,2020-01-22,"Korea, South",1,0,0


In [7]:
fig=px.choropleth(df_sort, locations='Country', locationmode='country names',color='Confirmed',animation_frame='Date')
# animation_frame will follow the order of column 'Date'
fig.update_layout(title_text='Global Spread of COVID-19')
# name graph
fig.show()

In [8]:
fig=px.choropleth(df, locations='Country', locationmode='country names',color='Deaths',animation_frame='Date')
fig.update_layout(title_text='Global Deaths of COVID-19')
fig.show()

## Task 3: 

### Visualizing how intensive the Covid-19 Transmission has been in each country by measuring new increased cases in last 24 hours

In [9]:
df_China=df[df.Country=='China']
df_China.head()

,Date,Country,Confirmed,Recovered,Deaths
9612,2020-01-22,China,548,28,17
9613,2020-01-23,China,643,30,18
9614,2020-01-24,China,920,36,26
9615,2020-01-25,China,1406,39,42
9616,2020-01-26,China,2075,49,56


In [10]:
df_China=df_China[['Date','Confirmed']]
df_China.head()

,Date,Confirmed
9612,2020-01-22,548
9613,2020-01-23,643
9614,2020-01-24,920
9615,2020-01-25,1406
9616,2020-01-26,2075


calculate the first derivation of confirmed column, save it in a new column

In [11]:
df_China['New Confirmed']=df_China['Confirmed'].diff()
df_China['Confirmed']=df_China['Confirmed'].astype(float)
# create a new column to show the diff
# you can choose to use Series or dataFrame
df_China.head()

,Date,Confirmed,New Confirmed
9612,2020-01-22,548.0,NaN
9613,2020-01-23,643.0,95.0
9614,2020-01-24,920.0,277.0
9615,2020-01-25,1406.0,486.0
9616,2020-01-26,2075.0,669.0


In [12]:
# type(df_China['Confirmed']) --> pandas.core.series.Series

draw a graph to show the change

In [13]:
fig=px.line(df_China, x='Date', y=['Confirmed','New Confirmed'])
fig.update_layout(title_text='New Confirmed cases of COVID-19')
#fig=px.line(df_China, x='Date', y='Confirmed')
#fig.add_scatter(x=df_China['Date'], y=df_China['Confirmed Deriv'],mode='lines',name='Confirmed Deriv')
fig.show()

In [14]:
df_China['New Confirmed'].max()

15136.0

# Task 3.2
## calculate the maximum new infections for each country

In [15]:
df_sort

,Date,Country,Confirmed,Recovered,Deaths
23229,2020-01-22,Japan,2,0,0
46725,2020-01-22,US,1,0,0
44322,2020-01-22,Taiwan*,1,0,0
45123,2020-01-22,Thailand,2,0,0
24297,2020-01-22,"Korea, South",1,0,0
...,...,...,...,...,...
37112,2020-10-14,Qatar,128603,125584,220
10946,2020-10-14,Congo (Kinshasa),10935,10306,281
36845,2020-10-14,Portugal,91193,54493,2117
9611,2020-10-14,Chile,485372,458073,13415


In [16]:
countries= list(df_sort['Country'].unique())
max_infection_rate=[]
for i in countries: 
  MIR=df[df.Country==i].Confirmed.diff().max()
  max_infection_rate.append(MIR)

## Task 3.3
### plot barchart to show maximum new infection in each country


In [17]:
df_MIR=pd.DataFrame()
df_MIR['Country']=countries
df_MIR['Max Infection Rate']=max_infection_rate
df_MIR.head()

,Country,Max Infection Rate
0,Japan,1762.0
1,US,77362.0
2,Taiwan*,27.0
3,Thailand,188.0
4,"Korea, South",851.0


In [18]:
px.bar(df_MIR,x='Country',y='Max Infection Rate',color='Country',title='Global Maximum New Infections',log_y=True)

# Task 4: 
See how National Lockdown began in 9th March 2020 impacts Covid 19 transmission in Italy

In [19]:
df_italy=df[df.Country=='Italy']

calculate the daily new infections in Italy

In [20]:
df_italy['New Infection']=df_italy.Confirmed.diff()
df_italy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Country,Confirmed,Recovered,Deaths,New Infection
22704,2020-01-31,Italy,2,0,0,NaN
22705,2020-02-01,Italy,2,0,0,0.0
22706,2020-02-02,Italy,2,0,0,0.0
22707,2020-02-03,Italy,2,0,0,0.0
22708,2020-02-04,Italy,2,0,0,0.0
...,...,...,...,...,...,...
22957,2020-10-10,Italy,349494,238525,36140,5724.0
22958,2020-10-11,Italy,354950,239709,36166,5456.0
22959,2020-10-12,Italy,359569,240600,36205,4619.0
22960,2020-10-13,Italy,365467,242028,36246,5898.0


In [21]:
italy_lockdown_start_date='2020-03-09'
italy_lockdown_after_a_month='2020-04-09'
fig=px.line(df_italy,x='Date',y='New Infection',title='Before and After Lockdown')
fig.add_shape(
    type='line',
    x0 =italy_lockdown_start_date,
    y0 =0,
    x1 =italy_lockdown_start_date,
    y1 = df_italy['New Infection'].max(),
    line=dict(color='red',width=2)
)
fig.add_annotation(
    x =italy_lockdown_start_date,
    y = df_italy['New Infection'].max(),
    text='Starting date of the lockdown'
)
fig.add_shape(
    type='line',
    x0 =italy_lockdown_after_a_month,
    y0 =0,
    x1 =italy_lockdown_after_a_month,
    y1 = df_italy['New Infection'].max(),
    line=dict(color='red',width=2)
)
fig.add_annotation(
    x =italy_lockdown_after_a_month,
    y = 0,
    text='Lockdown after a month'
)
fig.show()

# Task 5: See How National Lockdown Impacts COVID 19 Deaths rate in Italy

In [22]:
df_italy

,Date,Country,Confirmed,Recovered,Deaths,New Infection
22704,2020-01-31,Italy,2,0,0,NaN
22705,2020-02-01,Italy,2,0,0,0.0
22706,2020-02-02,Italy,2,0,0,0.0
22707,2020-02-03,Italy,2,0,0,0.0
22708,2020-02-04,Italy,2,0,0,0.0
...,...,...,...,...,...,...
22957,2020-10-10,Italy,349494,238525,36140,5724.0
22958,2020-10-11,Italy,354950,239709,36166,5456.0
22959,2020-10-12,Italy,359569,240600,36205,4619.0
22960,2020-10-13,Italy,365467,242028,36246,5898.0


calculate death rate

In [23]:
df_italy['New Deaths']=df_italy['Deaths'].diff()
df_italy['Confirmed']=df_italy['Confirmed'].astype(float)
df_italy


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Country,Confirmed,Recovered,Deaths,New Infection,New Deaths
22704,2020-01-31,Italy,2.0,0,0,NaN,NaN
22705,2020-02-01,Italy,2.0,0,0,0.0,0.0
22706,2020-02-02,Italy,2.0,0,0,0.0,0.0
22707,2020-02-03,Italy,2.0,0,0,0.0,0.0
22708,2020-02-04,Italy,2.0,0,0,0.0,0.0
...,...,...,...,...,...,...,...
22957,2020-10-10,Italy,349494.0,238525,36140,5724.0,29.0
22958,2020-10-11,Italy,354950.0,239709,36166,5456.0,26.0
22959,2020-10-12,Italy,359569.0,240600,36205,4619.0,39.0
22960,2020-10-13,Italy,365467.0,242028,36246,5898.0,41.0


plot a line chart to compare COVID 19 national lockdown impacts on spread of the virus and death rate

In [24]:
italy_lockdown_start_date='2020-03-09'
italy_lockdown_after_a_month='2020-04-09'
fig=px.line(df_italy, x='Date', y=['New Infection','New Deaths'],title='Lockdown Impact of COVID-19')
fig.add_shape(
    type='line',
    x0 =italy_lockdown_start_date,
    y0 =0,
    x1 =italy_lockdown_start_date,
    y1 = df_italy['New Infection'].max(),
    line=dict(color='green',width=2)
)
fig.add_annotation(
    x =italy_lockdown_start_date,
    y = df_italy['New Infection'].max(),
    text='Starting date of the lockdown'
)
fig.add_shape(
    type='line',
    x0 =italy_lockdown_after_a_month,
    y0 =0,
    x1 =italy_lockdown_after_a_month,
    y1 = df_italy['New Infection'].max(),
    line=dict(color='green',width=2)
)
fig.add_annotation(
    x =italy_lockdown_after_a_month,
    y = 0,
    text='Lockdown after a month'
)
fig.show()

But we hope to see the pattern of these two variables, whether they share any similiarities, so we need to normalize these two columns

In [25]:
df_italy['New Infection Norm']=df_italy['New Infection']/df_italy['New Infection'].max()
df_italy['New Deaths Norm']=df_italy['New Deaths']/df_italy['New Deaths'].max()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



plot the chart again

In [26]:
fig=px.line(df_italy,x='Date',y=['New Infection Norm','New Deaths Norm'],title='Lockdown Impact of COVID-19')
fig.add_shape(
    type='line',
    x0 =italy_lockdown_start_date,
    y0 =0,
    x1 =italy_lockdown_start_date,
    y1 = df_italy['New Infection Norm'].max(),
    line=dict(color='black',width=2)
)
fig.add_annotation(
    x =italy_lockdown_start_date,
    y = df_italy['New Infection Norm'].max(),
    text='Starting date of the lockdown'
)
fig.add_shape(
    type='line',
    x0 =italy_lockdown_after_a_month,
    y0 =0,
    x1 =italy_lockdown_after_a_month,
    y1 = df_italy['New Infection Norm'].max(),
    line=dict(color='orange',width=2)
)
fig.add_annotation(
    x =italy_lockdown_after_a_month,
    y = 0,
    text='Lockdown after a month'
)
fig.show()